In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
pd.set_option('display.max_row', 1200)
pd.set_option('display.max_columns', 100)

# 1950~2067년
dfDeathWoman = pd.read_excel('../../data/여성사망률추이.xlsx').set_index(['year']).iloc[:98,:100]
dfDeathMan = pd.read_excel('../../data/남성사망률추이.xlsx').set_index(['year']).iloc[:98,:100]
npDeathWoman = np.array(dfDeathWoman)
npDeathMan = np.array(dfDeathMan)

# 1950~2047년
dfDoctorData = pd.read_excel('../../data/doctorData.xlsx').set_index(['년도']).iloc[:98,:]
npRealDoctor = np.around(np.array(dfDoctorData[['의사수/계','의사수/남','의사수/여']])).T
npRealWorkDoctor = np.around(np.array(dfDoctorData['건강보험\n신고의사'])).T
npPassDoctor = np.around(np.array(dfDoctorData[['의대졸합격/남', '의대졸합격/여', '의전졸합격/남', '의전졸합격/여', '불합합격/남', '불합합격/여']]))
npDoctorExam = np.around(np.array(dfDoctorData[['의사고시 합격자수','의사고시 불합격자수']]))
npPopulation = np.array(dfDoctorData['추계인구'])
npPassDoctorRate = np.array(dfDoctorData[['의대졸비율/남', '의대졸비율/여', '의전졸비율/남', '의전졸비율/여', '불합비율/남', '불합비율/여']])
npCurrentDoctor = np.array(dfDoctorData[['의사수/계','의사수/남','의사수/여','의사수비율']])
npArmyDoctor = np.around(np.array(dfDoctorData[['군입대','군전역']]))

# 2011~2016년
dfDoctorAgeData = pd.read_excel('../../data/의사연령별분포.xlsx')
npDoctorAgeData = np.array(dfDoctorAgeData)[:,2:]

dfFirstDoctor = pd.read_excel('../../data/firstDoctor.xlsx')
npFirstDoctor = np.array(dfFirstDoctor)[:,1:-1]

# 1950~2020년 연령분포
dfKoreaAgePopRateData = pd.read_excel('../../data/koreaAgePopRateData.xlsx')
npKoreaAgePopRateDataMan = np.array(dfKoreaAgePopRateData[dfKoreaAgePopRateData['sex']=='MALE'])[:,2:]
npKoreaAgePopRateDataWoman = np.array(dfKoreaAgePopRateData[dfKoreaAgePopRateData['sex']=='FEMALE'])[:,2:]

In [5]:
dfPopulation = pd.DataFrame(np.around(npPopulation))
dfPopulation.index = range(1950, 2048)

popDoc = list(np.array(dfPopulation.T))
len(list(popDoc[0]))

98

In [24]:
npRealWorkDoctor

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,  60528,  63201,  65534,
        68143,  70355,  75714,  79046,  82137,  84544,  86761,  90710,
        92979,  95076,  97713, 100241, 102471, 105628, 107928,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0])

In [10]:
def clusterAgeModel(rate,st,end):
    npData = np.zeros(100)
    
    for i in range(st,end):
        npData[i] = (rate**(i-end))-1  
        
    sumValue = np.sum(npData)
    npRateData = npData / sumValue
    return npRateData

def makeAlivePerson(npData,year):
    
    deathRate = np.array([npDeathMan[year], npDeathWoman[year]])
    deadPerson = np.around((npData*deathRate))
    resultData = npData - deadPerson
    return [resultData, deadPerson]

def makeWorkPerson(npData,tuningSet):
    valueList = np.zeros([2,100])
    c1 = tuningSet[2][0]/((tuningSet[0][0]**(100-tuningSet[1][0]))-1)
    c2 = tuningSet[2][1]/((tuningSet[0][1]**(100-tuningSet[1][1]))-1)
    for i in range(100):
        result1 = ((tuningSet[0][0]**(i-tuningSet[1][0]))-1)*c1
        result2 = ((tuningSet[0][1]**(i-tuningSet[1][1]))-1)*c2
        valueList[0][i] = 0 if result1<0 else 1 if result1>1 else  result1
        valueList[1][i] = 0 if result2<0 else 1 if result2>1 else  result2

    retirePerson = np.around(npData*valueList)
    result = npData - retirePerson
    
    return [result, retirePerson]

def makeArrayUseModel(tuningList):
    model1 = clusterAgeModel(tuningList[0][0], tuningList[1][0], tuningList[2][0]) # 의대 남
    model2 = clusterAgeModel(tuningList[0][1], tuningList[1][1], tuningList[2][1]) # 의대 여
    model3 = clusterAgeModel(tuningList[0][2], tuningList[1][2], tuningList[2][2]) # 의전원 남
    model4 = clusterAgeModel(tuningList[0][3], tuningList[1][3], tuningList[2][3]) # 의전원 여
    model5 = clusterAgeModel(tuningList[0][4], tuningList[1][4], tuningList[2][4]) # 재시험 남
    model6 = clusterAgeModel(tuningList[0][5], tuningList[1][5], tuningList[2][5]) # 재시험 여
    
    resultData =  np.array([model1, model2, model3, model4, model5, model6])
    
    return resultData

def makeNewPerson(npData, tuningSet, npType=0):
    if npType==0:
        oldSize = 100
        yearSize = len(npData)
        modelSize = len(npData[0])

        #모델 적용 배열 98 x 6 x 100
        applyModelArray = np.zeros((yearSize,modelSize,oldSize))

        #신규인원 배열 98 x 2 x 100
        newPersonArray = np.zeros((yearSize,2,oldSize))

        modelAry = makeArrayUseModel(tuningSet)

        for i in range(yearSize):
            for j in range(modelSize):
                applyModelArray[i][j] = np.around(modelAry[j]*np.around(npData[i][j]))

                if j%2==0:
                    newPersonArray[i][0] += applyModelArray[i][j]
                else:
                    newPersonArray[i][1] += applyModelArray[i][j]

        resultData = newPersonArray

        return resultData
    
    else:
        oldSize = 100
        modelSize = 6

        #모델 적용 배열 6 x 100
        applyModelArray = np.zeros((modelSize,oldSize))

        #신규인원 배열 2 x 100
        newPersonArray = np.zeros((2,oldSize))

        modelAry = makeArrayUseModel(tuningSet)

        for i in range(modelSize):
            applyModelArray[i] = np.around(modelAry[i]*npData[i])

            if i%2==0:
                newPersonArray[0] += applyModelArray[i]
            else:
                newPersonArray[1] += applyModelArray[i]

        resultData = newPersonArray

        return resultData

def shiftOld(personArray):
    dataArray = np.zeros((2,len(personArray[0])))
    dataArray[0] = np.roll(personArray[0], 1)  
    dataArray[1] = np.roll(personArray[1], 1)
    dataArray[0][0] = 0
    dataArray[1][0] = 0
    
    return dataArray

def makeResultPersonArray(npData, tuningSet):
    #누적인원 배열
    sizeArray = list(np.shape(newPersonArray))
    yearSize = sizeArray[0]
    
    # 98 x 2 x 100 
    resultPersonArray = np.zeros(sizeArray)
    deadPersonArray = np.zeros(sizeArray)
    retirePersonArray = np.zeros(sizeArray)
    
    resultPersonArray[1][0] = np.around(npFirstDoctor[0].astype(np.double))
    resultPersonArray[1][1] = np.around(npFirstDoctor[1].astype(np.double))
    
    
    
    
    for i in range(2,yearSize):
        # 1살 올리기 / shiftData -> 2 x 100
        shiftData = shiftOld(resultPersonArray[i-1])
        # 사망률 적용 / aliveData -> 2 x 100
        aliveData = makeAlivePerson(shiftData, i)
        if i==20:
            print(aliveData)
        deadPersonArray[i] = aliveData[1]
        # 은퇴율 적용 / workData -> 2 x 100
        workData = makeWorkPerson(aliveData[0], tuningSet)
        retirePersonArray[i] = workData[1]
        # 최종 계산
        resultPersonArray[i] =  workData[0] + newPersonArray[i]

            
            
    return [resultPersonArray,newPersonArray,deadPersonArray,retirePersonArray]


def sumPeopleUseAge(npData):
    yearSize = len(npData)
    resultData = np.zeros([yearSize, 3, 12])
    
    for i in range(yearSize):
        for j in range(2):
            # 남/여 소계 계산
            resultData[i][j][0] = np.sum(npData[i][j])
            resultData[i][j][1] = np.sum(npData[i][j][0:30])
            resultData[i][j][2] = np.sum(npData[i][j][30:40])
            resultData[i][j][3] = np.sum(npData[i][j][40:50])
            resultData[i][j][4] = np.sum(npData[i][j][50:60])
            resultData[i][j][5] = np.sum(npData[i][j][60:])
            
            # 남/여 비율 계산
            for n in range(6):
                resultData[i][j][6+n] = resultData[i][j][n]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
        
        for j in range(6):
            # 합계 계산
            resultData[i][2][j] = resultData[i][0][j] + resultData[i][1][j]  
            # 합계 비율 계산
            resultData[i][2][6+j] = resultData[i][2][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            # 남/여 비율 계산
            resultData[i][0][6+j] = resultData[i][0][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0
            resultData[i][1][6+j] = resultData[i][1][j]/resultData[i][2][0] if resultData[i][2][0] > 0 else 0


    return resultData



def calculateCost(npData):   
    npSumData = np.concatenate((npData[0][:2],npData[1][:2],npData[2][:2],npData[3][:2],npData[4][:2],npData[5][:2]),axis=0)
    npResultData = np.abs(npDoctorAgeData - npSumData)
    return npResultData


def getCost(npData, tuningSet):
    newPerson = makeNewPerson(npData, tuningSet[0])
    resultPerson = makeResultPersonArray(newPerson,tuningSet[1])
    
    # sumData -> 43 x 3 x 12
    sumData = sumPeopleUseAge(resultPerson)
    costData = calculateCost(sumData)
    
    return [sumData, costData]

def makeDataFrame(npData):
    dfList = []
    startYear = 1950
    for i in range(4):
        for j in range(2):
            dfResult = pd.DataFrame(npData[i][0][j]).T
            indexList = []
            indexList.append(startYear)

            for k in range(1,98):
                dfResult = pd.concat([dfResult,pd.DataFrame(npData[i][k][j]).T])
                indexList.append(k+startYear)

            dfResult.index = indexList 
            dfList.append(dfResult)
            
    return dfList

def makeThousandPerDoctor(dfResultPerson, npPopulation):
    startYear = 1950
    dfResultPersonSum = dfResultPerson[2].sum(axis=1)
    npResultPersonSum = np.array(dfResultPersonSum)
    dfThousandPerDoctor = pd.DataFrame(npResultPersonSum/npPopulation*1000)
    dfThousandPerDoctor.index = range(startYear, 2048)
    return dfThousandPerDoctor

def makeFuturePerson(npBasicPopulation):
    yearSize = 98
    optSize = len(npBasicPopulation)
    npFuturePerson = np.zeros([yearSize,6])
    
    for i in range(optSize):
        npDoctorExam[i+76][0] = (npBasicPopulation[i][0]*0.94)+(npBasicPopulation[i][1]*0.94) #의대 합격자
        npDoctorExam[i+76][1] = (npBasicPopulation[i][0]*0.06)+(npBasicPopulation[i][1]*0.06) # 의대 불합격자 
        
        popSum = npBasicPopulation[i][0]+npBasicPopulation[i][1]+npDoctorExam[i+75][1]

        npPassDoctorRate[i+76][0] = (npBasicPopulation[i][0]*npBasicPopulation[i][2])/popSum #의대남자비율
        npPassDoctorRate[i+76][1] = (npBasicPopulation[i][0]*(1-npBasicPopulation[i][2]))/popSum #의대여자비율
        npPassDoctorRate[i+76][2] = (npBasicPopulation[i][1]*npBasicPopulation[i][3])/popSum #의전원남자비율
        npPassDoctorRate[i+76][3] = (npBasicPopulation[i][1]*(1-npBasicPopulation[i][3]))/popSum #의전원여자비율
        npPassDoctorRate[i+76][4] = npDoctorExam[i+75][1]*(npPassDoctorRate[i+76][0]+npPassDoctorRate[i+76][2])/popSum # 불합격남자비율
        npPassDoctorRate[i+76][5] = npDoctorExam[i+75][1]*(npPassDoctorRate[i+76][1]+npPassDoctorRate[i+76][3])/popSum # 불합격여자비율
        
        npFuturePerson[i+76][0] = np.around(popSum * npPassDoctorRate[i+76][0])
        npFuturePerson[i+76][1] = np.around(popSum * npPassDoctorRate[i+76][1])
        npFuturePerson[i+76][2] = np.around(popSum * npPassDoctorRate[i+76][2])
        npFuturePerson[i+76][3] = np.around(popSum * npPassDoctorRate[i+76][3])
        npFuturePerson[i+76][4] = np.around(popSum * npPassDoctorRate[i+76][4])
        npFuturePerson[i+76][5] = np.around(popSum * npPassDoctorRate[i+76][5])
        
    return npFuturePerson





In [11]:
def makeDoctorData(npData, tuningSet):
    npFuturePerson = makeFuturePerson(npData)
    npNewPassDoctor = npPassDoctor + npFuturePerson
    newPerson = makeNewPerson(npNewPassDoctor, tuningSet[0])
    resultData = makeResultPersonArray(newPerson,tuningSet[1])

    dfData = makeDataFrame(resultData)

    dfResultPerson = [dfData[0],dfData[1],dfData[0]+dfData[1]] # 의사수
    dfNewPerson = [dfData[2],dfData[3],dfData[2]+dfData[3]]    # 신규의사수
    dfDeadPerson = [dfData[4],dfData[5],dfData[4]+dfData[5]]   # 사망자수
    dfRetirePerson = [dfData[6],dfData[7],dfData[6]+dfData[7]] # 은퇴자수
    dfThousandPerDoctor = makeThousandPerDoctor(dfResultPerson, np.around(npPopulation)) # 1000명당 의사수
    dfPopulation = pd.DataFrame(np.around(npPopulation))
    dfPopulation.index = range(1950, 2048)
    dfPopulation
    
    return [dfResultPerson, dfNewPerson, dfDeadPerson, dfRetirePerson, dfThousandPerDoctor, dfPopulation]

In [12]:
# 77년 이전 신규의사수 추정
# 98 x 2 x 100 
resultPersonArray = np.zeros([27,2,100])
deadPersonArray = np.zeros([27,2,100])
retirePersonArray = np.zeros([27,2,100])
inArmy = np.zeros([27,2])
outArmy = np.zeros([27,2])
deadPerson = np.zeros([27,2])
retirePerson = np.zeros([27,2])

# 은퇴율 튜닝값
tuningSetRetireRate = [[1.2, 1.1],[30, 30],[1.8, 0.1]]

# 초기인원 입력
resultPersonArray[1][0] = np.around(npFirstDoctor[0].astype(np.double))
resultPersonArray[1][1] = np.around(npFirstDoctor[1].astype(np.double))

yearSize = 27

for i in range(2,yearSize):
    # 1살 올리기 / shiftData -> 2 x 100
    shiftData = shiftOld(resultPersonArray[i-1])
    # 사망률 적용 / aliveData -> 2 x 100
    aliveData = makeAlivePerson(shiftData, i)
    deadPersonArray[i] = aliveData[1]
    # 은퇴율 적용 / workData -> 2 x 100
    workData = makeWorkPerson(aliveData[0], tuningSetRetireRate)
    retirePersonArray[i] = workData[1]
    # 최종 계산
    resultPersonArray[i] =  workData[0] + newPersonArray[i]
    


NameError: name 'newPersonArray' is not defined

In [9]:
#의대수, 의전원수, 의대남비율, 의전원남비율 
npBasicPopulation = np.zeros([22,4])
for i in range(22):
    npBasicPopulation[i] = np.array([3400,50,0.6,0.6])
    
tuningSetAgeRate = [[0.05, 0.05, 0.05, 0.05, 0.6, 0.1],[25,25,29,29,26,26],[40, 40, 40, 40, 40, 40]]
tuningSetRetireRate = [[1.2, 1.1],[30, 30],[1.8, 0.1]]
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]

npFuturePerson = makeFuturePerson(npBasicPopulation)
npNewPassDoctor = npPassDoctor + npFuturePerson
newPerson = makeNewPerson(npNewPassDoctor, tuningSet[0])
resultData = makeResultPersonArray(newPerson,tuningSet[1])

dfData = makeDataFrame(resultData)

dfResultPerson = [dfData[0],dfData[1],dfData[0]+dfData[1]] # 의사수
dfNewPerson = [dfData[2],dfData[3],dfData[2]+dfData[3]]    # 신규의사수
dfDeadPerson = [dfData[4],dfData[5],dfData[4]+dfData[5]]   # 사망자수
dfRetirePerson = [dfData[6],dfData[7],dfData[6]+dfData[7]] # 은퇴자수
dfThousandPerDoctor = makeThousandPerDoctor(dfResultPerson, npPopulation) # 1000명당 의사수
    
npSumData = sumPeopleUseAge(resultData[0][61:67])
npSumData2 = sumPeopleUseAge(resultData[0])
npCalData = calculateCost(npSumData)

NameError: name 'newPersonArray' is not defined

In [74]:
def makeResultPersonArray(tuningSet):
    #누적인원 배열    # 98 x 2 x 100 
    sizeArray = [98,2,100]
    yearSize = sizeArray[0]
    resultPersonArray = np.zeros(sizeArray)
    deadPersonArray = np.zeros(sizeArray)
    retirePersonArray = np.zeros(sizeArray)
    npNewPerson = np.zeros(sizeArray)

    # 1952~1976 계산
    # npCurrentDoctor : 현재인력
    # npPassDoctor :  의사고시통과인력
    
    backPersonList = np.zeros([98,2]) # 복귀인력
    returnJobPersonList = np.zeros([98,2]) # 복직인력
    outArmyPersonList = np.zeros([98,2]) # 전역인력

    exitPersonList = np.zeros([98,2]) # 이탈인력
    deadPersonList = np.zeros([98,2]) # 사망인력
    joinArmyPersonlist = np.zeros([98,2]) # 입대인력 전년도 
    retirePersonList = np.zeros([98,2]) # 은퇴인력
    leavePersonList = np.zeros([92,2]) #  기타 이탈인력
    
    sumExitBackPersonList = np.zeros([98,2]) # 복귀인력 + 이탈인력
    
    
    # (N년)시험통과인력 = (N년)인력 - (N-1년)인력 - (N년)복귀인력(전역,입국,복직) + (N년)이탈인력(사망, 입대, 은퇴, 출국)
    
    resultPersonArray[1][0] = np.around(npFirstDoctor[0].astype(np.double))
    resultPersonArray[1][1] = np.around(npFirstDoctor[1].astype(np.double))
    
    
    for i in range(2,27):
        # 1살 올리기 / shiftData -> 2 x 100
        shiftData = shiftOld(resultPersonArray[i-1])
        # 사망률 적용 / aliveData -> 2 x 100
        aliveData = makeAlivePerson(shiftData, i)
        deadPersonArray[i] = aliveData[1]
        deadPersonList[i][0] = deadPersonArray[i][0].sum() # 사망남성 수
        deadPersonList[i][1] = deadPersonArray[i][1].sum() # 사망여성 수
        # 은퇴율 적용 / workData -> 2 x 100
        workData = makeWorkPerson(aliveData[0], tuningSet[1])
        retirePersonArray[i] = workData[1]
        retirePersonList[i][0] = retirePersonArray[i][0].sum() # 은퇴남성 수
        retirePersonList[i][1] = retirePersonArray[i][1].sum() # 은퇴여성 수
        
        # 복귀인원 - 이탈인원 계산
        sumExitBackPersonList[i][0] = npCurrentDoctor[i][0] - npCurrentDoctor[i-1][0] # 남성
        sumExitBackPersonList[i][1] = npCurrentDoctor[i][1] - npCurrentDoctor[i-1][1] # 여성
        
        if sumExitBackPersonList[i][0]>=0:        
            # 이탈인력 계산
            exitPersonList[i][0] = deadPersonList[i][0] + retirePersonList[i][0] + npArmyDoctor[i][0]
            # 복귀인력 계산
            backPersonList[i][0] = sumExitBackPersonList[i][0] - exitPersonList[i][0]
            # 복직인력 계산
            returnJobPersonList[i][0] = backPersonList[i][0] - npArmyDoctor[i][1]
        else :
            # 복귀인력 계산
            backPersonList[i][0] = outArmyPersonList[i][0]
            # 이탈인력 계산
            exitPersonList[i][0] = sumExitBackPersonList[i][0] -  backPersonList[i][0]
            # 기타 이탈인력 계산
            leavePersonList[i][0] = exitPersonList[i][0] - (deadPersonList[i][0] + retirePersonList[i][0] + npArmyDoctor[i][0])
            
            
        if sumExitBackPersonList[i][1]>=0:        
            # 이탈인력 계산
            exitPersonList[i][1] = deadPersonList[i][1] + retirePersonList[i][1]
            # 복귀인력 계산
            backPersonList[i][1] = sumExitBackPersonList[i][1] - exitPersonList[i][1]
            # 복직인력 계산
            returnJobPersonList[i][1] = backPersonList[i][1]
            
        else :
            # 이탈인력 계산
            exitPersonList[i][1] = sumExitBackPersonList[i][1] -  backPersonList[i][1]
            # 기타 이탈인력 계산
            leavePersonList[i][1] = exitPersonList[i][1] - (deadPersonList[i][1] + retirePersonList[i][1])
            
            
        # 시험통과인력
        # 남성
        npPassDoctor[i][0] = npCurrentDoctor[i][1] - npCurrentDoctor[i-1][1] - backPersonList[i][0] + exitPersonList[i][0]
        # 여성
        npPassDoctor[i][1] = npCurrentDoctor[i][2] - npCurrentDoctor[i-1][2] - backPersonList[i][1] + exitPersonList[i][1]
        
        npNewPerson[i] = makeNewPerson(npPassDoctor[i], tuningSet[0],1)
        
        # 최종 계산
        resultPersonArray[i] =  workData[0] +  npNewPerson[i]

    # 1977~2047 계산
    
    return [sumExitBackPersonList,npPassDoctor,npNewPerson,deadPersonArray,retirePersonArray,backPersonList,exitPersonList,returnJobPersonList,leavePersonList]


In [75]:
tuningSetAgeRate = [[0.05, 0.05, 0.05, 0.05, 0.6, 0.1],[25,25,29,29,26,26],[40, 40, 40, 40, 40, 40]]
tuningSetRetireRate = [[1.2, 1.1],[30, 30],[1.8, 0.1]]
tuningSet = [tuningSetAgeRate, tuningSetRetireRate]

result = makeResultPersonArray(tuningSet)

In [76]:
pd.DataFrame(result[0])

,0,1
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,5435.0,4838.0
6,288.0,230.0
7,345.0,292.0
8,337.0,268.0
9,209.0,123.0


In [68]:
pd.DataFrame(result[1][5])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-225.0,-11.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4029.0,-201.0,-10.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dfDeadPerson[2].sum(axis=1)

In [ ]:
pd.DataFrame(dfResultPerson[0])

In [ ]:
pd.DataFrame(npCalData)